# Adrien HANS & Tanguy JEANNEAU

# Entropie et génération de mots de passe
* Nous considérons un générateur de mots de passe qui consiste à prendre **8 lettres consécutives dans un texte** (sans se soucier des espaces).
* Nous faisons l'hypothèse que le texte n'est composé que des 26 lettres de l'alphabet, sans majuscules ni accents
* Nous proposons de calculer l'entropie d'une telle source, en fonction du modèle probabiliste considéré, et de calculer le temps moyen nécessaire pour trouver un mot de passe à partir de ce modèle. 

**Merci de commenter vos réponses**

In [1]:
import numpy as np
from numpy import genfromtxt
from pandas import read_csv
import pandas as pd
import time

#### Modèle monogramme (une lettre)
* On récupére des données composées de [lettre,frequence d'apparition de la lettre] (voir fichier csv pour [comma-separated-value](https://en.wikipedia.org/wiki/Comma-separated_values)) 

In [2]:
monogramme = read_csv('monogramme.csv')
freq_mono = (monogramme['frequency']).values
letters_mono = (monogramme['letters']).values

Q: Quelles sont les 5 lettres les plus représentées ?

In [3]:
monogramme.head()

,letters,frequency
0,E,0.1776
1,S,0.0823
2,A,0.0768
3,N,0.0761
4,T,0.0730


`monogramme` trié de la plus grande fréquence à la plus faible. 
Il faut donc seulement prendre les 5 premiers élèments de `letters_mono`

In [4]:
letters_mono[:5]

array(['E', 'S', 'A', 'N', 'T'], dtype=object)

Voici une fonction qui calcule l'entropie à partir d'un vecteur constitué de probabilités empiriques (note, je vous épargne le codage de cette fonction, car en moyenne vous passerez 15mn à gérer le cas `freq_bis==0`, mais il n'est pas inutile de bien comprendre ces 4 lignes).

In [5]:
def entropie(freq):
    freq_bis = np.copy(freq)
    freq_bis[freq_bis==0] = 1.0
    ent = -np.sum(freq_bis*np.log2(freq_bis))
    return ent

Q: en utilisant ce modèle probabiliste pour générer un mot de passe, quelle est l'entropie d'un mot de passe de 8 lettres ?

On suppose que l'apparition de lettres successives est indépendante, d'où : $H(X,Y)=H(X)+H(Y)$ 

Et donc l'entropie d'un mod de passe de 8 lettres est égale à 8 fois celui d'une lettre : 

In [6]:
#Pour 8 lettres : 
ent = entropie(freq_mono)*8
print("L'entropie d'un mot de passe de 8 lettres est : ", ent, " bits")

L'entropie d'un mot de passe de 8 lettres est :  31.676242429778334  bits


Q: A l'aide de la fonction `np.random.choice()`, calculer le temps nécessaire en secondes pour tirer 100 000 mots de passes en utilisant ce générateur ?

In [7]:
def generate_pwds(n, letters=letters_mono, freq=freq_mono, i=8):
    t = time.time()
    a = np.random.choice(letters_mono, size=(int(n),i), p=freq_mono)
    t_s = time.time() - t
    print('generated {:.2e} passwords in {:.3f} s'.format(n, t_s))
    return a, t_s

_, t_1e5 = generate_pwds(1e5)

generated 1.00e+05 passwords in 0.077 s


Tandis qu'avec une boucle `for` :

In [8]:
t = time.time()
for i in range(100000):
    np.random.choice(letters_mono, size=(8), p=freq_mono)
t_100000 = time.time() - t
print(t_100000, 's')

8.036882162094116 s


**Le plus rapide est donc assez largement de définir la taille voulue directement dans `np.random.choice` plutôt que d'utiliser une boucle `for`**

Nous definissons l'"entropie du devin" G (guessing entropie) comme le **nombre moyen d'essais successif nécessaires pour trouver un mot de passe à partir de notre générateur**.
On peut montrer que $G\geq H/4+1$ où $H$ est l'entropie de la source (voir [ce papier](./Password_Entropy_and_Password_Quality.pdf) )

Q: calculer $G$ pour ce modèle

In [9]:
def compute_G(H):
    G = (2**H)/4 +1
    print('G = {:.2e}'.format(G))
    return G

G = compute_G(ent)

G= 8.58e+08


Q: combien de temps cela prendra-t-il en utilisant le générateur codé précédemment ? (en minutes)

In [10]:
_, t_1e4 = generate_pwds(1e4)
_, t_1e5 = generate_pwds(1e5)
_, t_1e6 = generate_pwds(1e6)
_, t_1e7 = generate_pwds(1e7)

generated 1.00e+04 passwords in 0.005 s
generated 1.00e+05 passwords in 0.065 s
generated 1.00e+06 passwords in 0.810 s
generated 1.00e+07 passwords in 8.202 s


**On observe que le temps nécessaire pour générer i mots de passe de 8 lettres en utilisant le générateur codé précedemment est linéaire en i.** 

In [11]:
def compute_time(G, t, nb_t):
    tG = (G/(nb_t)) * t / 60
    print('Temps nécessaire pour générer G mots de passe : {:.2f} min'.format(tG))
    
compute_time(G, t_1e6, 1e6)

Temps nécessaire pour générer G mots de passe : 11.58 min


On propose maintenant d'utiliser un modèle plus évolué qui est construit à partir de la probabilité conjointe de deux lettres successives (bigramme)

In [12]:
bigramme = read_csv('bigramme.csv')
freq_bi = (bigramme['frequency']).values
letters_bi = (bigramme['letters']).values

Q: Quelles sont les 5 couples de lettres les plus représentés ?

In [13]:
bigramme.head(5)

,letters,frequency
0,AA,0.000252
1,AB,0.001760
2,AC,0.003482
3,AD,0.001960
4,AE,0.000200


`bigramme` n'est pas trié en fonction de la fréquence.

In [14]:
bigramme.sort_values('frequency', ascending=False, inplace=True)

In [15]:
bigramme.head()

,letters,frequency
122,ES,0.023809
117,EN,0.021248
82,DE,0.019570
290,LE,0.018845
357,NT,0.017009


Maintenant `bigramme` est trié en fonction de la fréquence.

Q: en utilisant ce modèle probabiliste pour générer un mot de passe, quelle est l'entropie d'un mot de passe de 8 lettres ?

**On suppose que chaque couple est indépendant et donc que pour créer un mot de passe de 8 lettres il faut générer 4 couples de lettres, d'où l'entropie :**

In [16]:
ent_bi = 4*entropie(freq_bi)
print("L'entropie d'un mot de passe de 8 lettres en utilisant monogramme est : ", ent, " bits")
print("L'entropie d'un mot de passe de 8 lettres en utilisant bigramme est : ", ent_bi, " bits")

L'entropie d'un mot de passe de 8 lettres en utilisant monogramme est :  31.676242429778334  bits
L'entropie d'un mot de passe de 8 lettres en utilisant bigramme est :  30.14226404646422  bits


Q: Pourquoi cette entropie est-elle inférieure à celle du modèle construit sur des monogrammes ?

**Dans le cas général : $H(H,Y)\leq H(X) + H(Y)$  et $H(H,Y)= H(X) + H(Y)$ si et seulement si X et Y indépendants. 
Donc puisqu'on introduit une dépendance dans ce modèle avec `bigramme`, il est logique que l'entropie soit réduite.**

Q: A l'aide de la fonction `np.random.choice()`, calculer le temps nécessaire en secondes pour tirer 100 000 mots de passes en utilisant ce générateur ?

In [17]:
_, t_1e5_bi = generate_pwds(1e5, freq=freq_bi, letters=letters_bi, i=4)

generated 1.00e+05 passwords in 0.038 s


Q: calculer G pour ce modèle

In [18]:
G_bi = compute_G(ent_bi)

G= 2.96e+08


Q: combien de temps cela prendra-t-il en utilisant le générateur codé précédemment ? (en minutes)

In [19]:
_, t_1e4 = generate_pwds(1e4,freq=freq_bi, letters=letters_bi, i=4)
_, t_1e5 = generate_pwds(1e5,freq=freq_bi, letters=letters_bi, i=4)
_, t_1e6 = generate_pwds(1e6,freq=freq_bi, letters=letters_bi, i=4)
_, t_1e7 = generate_pwds(1e7,freq=freq_bi, letters=letters_bi, i=4)

generated 1.00e+04 passwords in 0.004 s
generated 1.00e+05 passwords in 0.043 s
generated 1.00e+06 passwords in 0.499 s
generated 1.00e+07 passwords in 4.174 s


**On observe que le temps nécessaire pour générer i mots de passe de 8 lettres en utilisant le modèle avec bigramme et le  générateur codé précedemment est linéaire en i.** 

In [20]:
compute_time(G_bi, t_1e6, 1e6)

Temps nécessaire pour générer G mots de passe : 2.46 min


Q: si maintenant on change de stratégie et on tire aléatoirement chaque lettre de l'alphabet de façon uniforme, quelle est l'entropie de ce générateur ?

On tire aléatoirement chaque lettre de l'alphabet de façon uniforme, d'où ici la frequence de chaque lettre associée :

In [21]:
freq_rnd = [1/26]*26

On utilise les mêmes hypothèses d'indépendance et les mêmes fonctions que précedemment :

In [22]:
#Pour 8 lettres : 
ent_rnd = entropie(freq_rnd)*8
print("L'entropie d'un mot de passe de 8 lettres en tirant aléatoirement chaque lettre est : ", ent_rnd, " bits")

L'entropie d'un mot de passe de 8 lettres en tirant aléatoirement chaque lettre est :  37.603517745128734  bits


Q: A l'aide de la fonction `np.random.choice()`, calculer le temps nécessaire en secondes pour tirer 100 000 mots de passes en utilisant ce générateur ?

In [23]:
_, _ = generate_pwds(1e5, letters=letters_mono, freq=freq_rnd, i=8)

generated 1.00e+05 passwords in 0.101 s


Q: calculer G pour ce modèle

In [24]:
G_rnd = compute_G(ent_rnd)

G= 5.22e+10


Q: combien de temps cela prendra-t-il en utilisant le générateur codé précédemment ? (en minutes)

**On sait déjà avec les questions précédentes, que le générateur de mots de passe est de compléxité linéaire en le nombre de mots de passe à générer, d'où :**

In [25]:
_, t_1e6 = generate_pwds(1e6, letters=letters_mono, freq=freq_rnd, i=8)
compute_time(G_rnd, t_1e6, 1e6)

generated 1.00e+06 passwords in 0.860 s
Temps nécessaire pour générer G mots de passe : 748.04 min


#### Conclure sur les liens entre la sécurité d'un générateur de mot de passe et le modèle probabilistique considéré

**Nous avons observé que plus on a d'informations sur la distribution des mots de passe, plus l'entropie $H$ et 'l'entropie du devin' $G$ diminue, et donc moins il faudra de tentative pour trouver le bon mot de passe.**

**De même, puisque nous avons observé que le générateur de mots de passe est de complexité linéaire en le nombre de mots de passe à générer, il faudra moins de temps pour trouver le bon.** 

Tout cela était logique avec la formule : 
\begin{align*}
H(X,Y) \leq H(X) + H(Y)
\end{align*}

avec 
\begin{align*}
H(X,Y)=H(X)+H(Y)
\end{align*}

si et seulement si X et Y indépendants. 

**Ainsi, trouver des dépendances entre les lettres ou bien des informations quant à leur fréqeunce d'apparition permet de diminuer l'entropie $H$, l'entropie du devin $G$ et le temps mis pour trouver le mot de passe.** 